In [10]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings, HuggingFaceHubEmbeddings
from langchain.vectorstores import Chroma
from openai import OpenAI
from dotenv import load_dotenv
import os

In [3]:
load_dotenv()
HF_TOKEN = os.getenv("HF_TOKEN")
OR_TOKEN = os.getenv("OR_TOKEN")

In [5]:
pdf_path='resume.pdf'
loader=PyPDFLoader(pdf_path)
content=loader.load()

In [6]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=256, chunk_overlap=50)
chunks=text_splitter.split_documents(content)

In [11]:
embeddings = HuggingFaceHubEmbeddings(
    repo_id="BAAI/bge-base-en-v1.5",
    huggingfacehub_api_token=HF_TOKEN
)

C:\Users\kanha\AppData\Local\Temp\ipykernel_17980\633541830.py:1: LangChainDeprecationWarning: The class `HuggingFaceHubEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEndpointEmbeddings``.
  embeddings = HuggingFaceHubEmbeddings(
c:\Users\kanha\Desktop\MERag\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
vectorstore=Chroma.from_documents(chunks,embeddings)

In [15]:
retriever = vectorstore.as_retriever(search_type="mmr", search_kwargs={"k": 3})

In [24]:
query = "CGPA"
docs_rel = retriever.get_relevant_documents(query)

c:\Users\kanha\Desktop\MERag\venv\lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


In [25]:
prompt = f"""
<|system|>>
You are an AI Assistant that follows instructions extremely well.
Please be truthful and give direct answers. Please tell 'I don't know' if user query is not in context
</s>
<|user|>
{query}
</s>
<|assistant|>
"""

In [26]:
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=OR_TOKEN,
)

completion = client.chat.completions.create(
    extra_headers={
        "HTTP-Referer": "<YOUR_SITE_URL>",  # Optional
        "X-Title": "<YOUR_SITE_NAME>",      # Optional
    },
    model="nvidia/llama-3.1-nemotron-nano-8b-v1:free",
    messages=[
        {"role": "user", "content": prompt}
    ]
)

In [27]:
print(completion.choices[0].message.content)

I don't know. Could you please specify which GPA you're interested in? This could be a high school GPA, a college GPA, or even a company's internal GPA program.
